In [3]:
from datasets import load_dataset, load_dataset_builder  # pip install datasets

In [11]:
dataset_name = "wikitext"
dataset_config = "wikitext-103-raw-v1"

In [18]:
dataset = load_dataset(dataset_name, dataset_config)

Dataset wikitext downloaded and prepared to C:\Users\Luke A Sargen\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00,  7.78it/s]


In [71]:
splits = ['train', 'validation', 'test']
filenames = [f"{dataset_config}-{split}.txt" for split in splits]
for split, filename in zip(splits, filenames):
    print(len(dataset[split]))
    with open(filename, "w+", encoding="UTF-8") as f:
        f.writelines(dataset[split]["text"])

1801350
3760
4358


In [112]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [113]:
test_str = "Tripled up to 99,999, and that's only the first round."

In [114]:
tokenizer = Tokenizer(models.BPE())
pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Digits(individual_digits=True),
    pre_tokenizers.ByteLevel(add_prefix_space=False)
])
tokenizer.pre_tokenizer = pre_tokenizer

In [115]:
trainer = trainers.BpeTrainer(vocab_size=2048, special_tokens=["<|endoftext|>"])
tokenizer.train(files=filenames[1:], trainer=trainer)

In [116]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [118]:
from transformers import PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    model_max_length=512,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)
wrapped_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=2048, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>'})

In [119]:
wrapped_tokenizer.save_pretrained("tokenizer-fast")

('tokenizer-fast\\tokenizer_config.json',
 'tokenizer-fast\\special_tokens_map.json',
 'tokenizer-fast\\tokenizer.json')

In [122]:
loaded_tokenizer = PreTrainedTokenizerFast.from_pretrained("tokenizer-fast")

In [121]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output.tokens)

['H', 'ell', 'o', ',', 'Ġy', "'", 'all', '!', 'ĠH', 'ow', 'Ġare', 'Ġyou', 'Ġ', 'ð', 'Ł', 'ĺ', 'ģ', 'Ġ', '?']


In [126]:
output = loaded_tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output)
decode = tokenizer.batch_decode(output)
print(decode)

[40, 422, 79, 12, 425, 7, 376, 1, 293, 297, 417, 1520, 171, 168, 203, 196, 173, 171, 31]


AttributeError: 'tokenizers.Tokenizer' object has no attribute 'batch_decode'